---
#### 모델링을 위한 데이터셋 만들기
---

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme(style="whitegrid", palette="pastel", 
              rc={"font.size": 9, "axes.titlesize": 9, "axes.labelsize": 9,
              "xtick.labelsize": 9, "ytick.labelsize": 9})  

import warnings
warnings.filterwarnings(action='ignore')  ## 경고문구 생략 

import koreanize_matplotlib               ## 한글사용 

#data loading
data_path = 'kaggle/input/competitive-data-science-predict-future-sales/'
sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

---
#### feature engineering 
    1. 변수명 한글화 
    2. downcasting 
    3. 데이터 조합 (월ID x 상점ID x 상품ID)
    4. 파생변수 생성&추가 
    5. 테스트 데이터 결합 
    6. 기타 데이터 병합 --> 최종 데이터셋 
    7. train/valid/test 데이터 분리 

In [2]:
print('sales_train: ', sales_train.columns)
print('shops: ', shops.columns)
print('items: ', items.columns)
print('item_categories: ', item_categories.columns)
print('test: ', test.columns)

sales_train:  Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day'],
      dtype='object')
shops:  Index(['shop_name', 'shop_id'], dtype='object')
items:  Index(['item_name', 'item_id', 'item_category_id'], dtype='object')
item_categories:  Index(['item_category_name', 'item_category_id'], dtype='object')
test:  Index(['ID', 'shop_id', 'item_id'], dtype='object')


In [3]:
#1.편의를 위해 변수명 한글화 
sales_train = sales_train.rename(columns = {'date': '날짜', 
                                            'date_block_num': '월ID', 
                                            'shop_id': '상점ID', 
                                            'item_id': '상품ID', 
                                            'item_price': '판매가', 
                                            'item_cnt_day': '판매량'})

shops = shops.rename(columns = {'shop_name': '상점명', 'shop_id': '상점ID' })
items = items.rename(columns = {'item_name': '상품명', 'item_id':'상품ID', 'item_category_id':'상품분류ID' })
item_categories = item_categories.rename(columns = {'item_category_name': '상품분류명', 'item_category_id': '상품분류ID'})
test = test.rename(columns = {'shop_id': '상점ID', 'item_id': '상품ID' })

In [4]:
print('sales_train: ', sales_train.columns)
print('shops: ', shops.columns)
print('items: ', items.columns)
print('item_categories: ', item_categories.columns)
print('test: ', test.columns)

sales_train:  Index(['날짜', '월ID', '상점ID', '상품ID', '판매가', '판매량'], dtype='object')
shops:  Index(['상점명', '상점ID'], dtype='object')
items:  Index(['상품명', '상품ID', '상품분류ID'], dtype='object')
item_categories:  Index(['상품분류명', '상품분류ID'], dtype='object')
test:  Index(['ID', '상점ID', '상품ID'], dtype='object')


In [5]:
#2.Downcasting (메모리 절약)
def downcast(df, verbose=True): 
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns : 
        dtype_name = df[col].dtype.name
        if dtype_name == 'object': 
            pass
        elif dtype_name == 'bool': 
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():  
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else: 
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: 
        print('{:.1f}% compressed'.format(100 * (start_mem - end_mem) / start_mem))

    return df 

In [6]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df: 
    df = downcast(df)

54.2% compressed
38.5% compressed
54.2% compressed
39.8% compressed
70.8% compressed


In [7]:
#3. 데이터 조합 (월ID x 상점ID x 상품ID) : product() 함수 사용 
#전체 월/상점/상품에 대해 판매량을 표기한 테이블 생성(판매=0인 경우도 생략하지 말고 표시)

from itertools import product

train = []
#월 x 상점 x 상품 ID별 조합 생성 
for i in sales_train['월ID'].unique() : 
    all_shop = sales_train.loc[sales_train['월ID'] == i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID'] == i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

#list type(train) --> DataFrame
idx_features = ['월ID', '상점ID', '상품ID'] #기준변수
train = pd.DataFrame(np.vstack(train), columns = idx_features)

train 

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


In [8]:
#3-1. 판매량 추가 
group = sales_train.groupby(idx_features).agg({'판매량':'sum'})
group = group.reset_index()
group = group.rename(columns = {'판매량':'월간판매량'})

group

,월ID,상점ID,상품ID,월간판매량
0,0,0,32,6
1,0,0,33,3
2,0,0,35,1
3,0,0,43,1
4,0,0,51,2
...,...,...,...,...
1609119,33,59,22087,6
1609120,33,59,22088,2
1609121,33,59,22091,1
1609122,33,59,22100,1


In [9]:
#3-2. train과 group 병합 
# train은 모든 월/상점/상품ID의 조합(약 11백만개), group은 월간판매량이 존재하는 것만(약 160만개)
train = train.merge(group, on = idx_features, how = 'left')
train

,월ID,상점ID,상품ID,월간판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10913845,33,21,7635,NaN
10913846,33,21,7638,NaN
10913847,33,21,7640,NaN
10913848,33,21,7632,NaN


In [10]:
## cf. 불필요한 데이터셋 제거 (memory save)
import gc 
del group 
gc.collect()

20

In [11]:
#4.최종 데이터셋 생성 
## train + test dataset 병합 & shops/items/item_categories 데이터 병합 
test

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


In [12]:
## test 데이터는 2015년 11월 분 --> 월ID = 34
## ID는 불필요하므로 drop 
## 결측치는 0으로 대체 

test['월ID'] = 34

all_data = pd.concat([train, test.drop('ID', axis=1)], 
                      ignore_index = True, 
                      keys = idx_features)

all_data = all_data.fillna(0)

all_data

,월ID,상점ID,상품ID,월간판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


In [13]:
## shops/items/item_categories 병합
## 메모리 절약위한 daoncasting & gc 

all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

all_data = downcast(all_data)

del shops, items, item_categories
gc.collect() 

31.1% compressed


0

In [14]:
all_data

,월ID,상점ID,상품ID,월간판매량,상점명,상품명,상품분류ID,상품분류명
0,0,59,22154,1,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,0,59,2552,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,0,59,2554,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
3,0,59,2555,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства
4,0,59,2564,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео
...,...,...,...,...,...,...,...,...
11128045,34,45,18454,0,"Самара ТЦ ""ПаркХаус""",СБ. Союз 55,55,Музыка - CD локального производства
11128046,34,45,16188,0,"Самара ТЦ ""ПаркХаус""",Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры
11128047,34,45,15757,0,"Самара ТЦ ""ПаркХаус""",НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства
11128048,34,45,19648,0,"Самара ТЦ ""ПаркХаус""",ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD


In [15]:
## 불필요한 변수 ~ 상점명/상품명/상품분류명 제거 
all_data = all_data.drop(['상점명', '상품명', '상품분류명'], axis=1)
all_data

,월ID,상점ID,상품ID,월간판매량,상품분류ID
0,0,59,22154,1,37
1,0,59,2552,0,58
2,0,59,2554,0,58
3,0,59,2555,0,56
4,0,59,2564,0,59
...,...,...,...,...,...
11128045,34,45,18454,0,55
11128046,34,45,16188,0,64
11128047,34,45,15757,0,55
11128048,34,45,19648,0,40


In [16]:
#5. data split 
## train : 2013.01 ~ 2015.09 / valid : 2015.10 / test : 2015.11 
## 월간판매량 20 이상은 제외하기로 함 

## features 
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간판매량'], axis=1)

X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간판매량'], axis=1)

X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간판매량'], axis=1)

## target
y_train = all_data[all_data['월ID'] < 33]['월간판매량']
y_train = y_train.clip(0,20)

y_valid = all_data[all_data['월ID'] == 33]['월간판매량']
y_valid = y_valid.clip(0,20)

In [17]:
del all_data
gc.collect()

0

#### Data set completed 
---

In [18]:
X_train.to_csv('temp_data/X_train.csv', index=False)
X_valid.to_csv('temp_data/X_valid.csv', index=False)
X_test.to_csv('temp_data/X_test.csv', index=False)

y_train.to_csv('temp_data/y_train.csv', index=False)
y_valid.to_csv('temp_data/y_valid.csv', index=False)